In [3]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
from datetime import datetime
import calendar
import itertools

EMPLOYER_MASTER = pd.read_csv("Employer master.csv")
PAYMENT_TRANSACTIONS = pd.read_csv("Payment transactions.csv")
START_DATE = datetime.strptime("2018-01","%Y-%m")
END_DATE = datetime.strptime("2018-12","%Y-%m")

def format_datetime(dt):
    return datetime.strptime(dt, "%Y-%m-%d").strftime("%d/%m/%Y")

def extract_year(dt):
    return datetime.strptime(dt, "%Y-%m").year

def extract_month(dt):
    return datetime.strptime(dt, "%Y-%m").month

def get_distinct_tier_list():
    '''
        This function will list the type of tier
    '''
    distinct_tier_list = list(set(EMPLOYER_MASTER["Tier"]))
    distinct_tier_list.sort()
    return distinct_tier_list

def date_range():
    '''
        This function will return a list of year-month,
        a list of first days for each month,
        and a list of last days for each month 
        between start date and end date
    '''
    # Ensure start date is earlier than end date
    if START_DATE > END_DATE:
        raise Exception("Please enter the correct start date and end date!")
    else:
        month_range = list(set([x for x in pd.date_range(START_DATE,END_DATE).strftime("%Y-%m")]))
        month_range.sort()
        month_first_date = [date(extract_year(x), extract_month(x), day = 1).strftime("%Y-%m-%d") for x in month_range]
        month_end_date = [date(extract_year(x), extract_month(x), 
                               day = calendar.monthrange(extract_year(x),extract_month(x))[1]).strftime("%Y-%m-%d") 
                          for x in month_range]
        return month_range, month_first_date, month_end_date

def tier_and_month_end_date_pairs():
    '''
        This function will generate the Tier column and the Month End Date column in the output
    '''
    month_end_date = [format_datetime(x) for x in date_range()[2]]
    # Produce all ordered pairs of Tier and Month End Date
    all_pairs = list(itertools.product(get_distinct_tier_list(),month_end_date))
    # Generate the Tier column
    tier_list = [pair[0] for pair in all_pairs]
    # Generate the Month End Date column
    month_end_date_list = [pair[1] for pair in all_pairs]
    return tier_list, month_end_date_list

def payments():
    '''
        This function will generate the Num Payments column and Amount Of Payments column in the output
    '''
    num_payments_list = []
    amount_of_payments_list = []
    for tier in get_distinct_tier_list():
        # Generate dataframe within one type of tier from Employer Master dataset
        tier = EMPLOYER_MASTER.loc[EMPLOYER_MASTER["Tier"] == tier]
        # Find all Employer_No within the specified tier from Employer Master dataset
        employer_num_list = list(set(tier["Employer_No"]))
        # Generate dataframe with the specified Employer_No and tier from Payment Transactions dataset
        transactions = PAYMENT_TRANSACTIONS.loc[PAYMENT_TRANSACTIONS["Employer_No"].isin(employer_num_list)]
        # Generate additional Formatted_Cash_Received_Date columns for Payment Transactions dataset
        transactions["Formatted_Cash_Received_Date"] = pd.to_datetime(transactions["Cash_Received_Date"], format = "%d/%m/%Y")
        # Genarate monthly number of payments and amount of payments
        for j in range(len(date_range()[0])):
            transaction_monthly = transactions.loc[(transactions["Formatted_Cash_Received_Date"] >= date_range()[1][j]) & 
                                                   (transactions["Formatted_Cash_Received_Date"] <= date_range()[2][j])]
            num_payments_list.append(len(transaction_monthly))
            amount_of_payments_list.append("%.2f" % transaction_monthly["Total_Amt"].sum())
    return num_payments_list, amount_of_payments_list

def new_employers_list():
    '''
        This function will generate the New Employers column in the output
    '''
    new_employers_list = []
    for tier in get_distinct_tier_list():
        for i in range(len(date_range()[0])):
            # Generate dataframe within one type of tier from Employer Master dataset
            records = EMPLOYER_MASTER.loc[EMPLOYER_MASTER["Tier"] == tier]
            # Find the earlist employer account record within the specified tier from Employer Master dataset
            records_distinct_employers = records.drop_duplicates(subset = "Employer_No", keep = "first", inplace = False)
            # Check whether the employer account is first created at current month
            new_employers = records_distinct_employers.loc[
                            (records_distinct_employers["Effective_From"] >= date_range()[1][i]) &
                            (records_distinct_employers["Effective_From"] <= date_range()[2][i])]
            # Count new employer accounts at current month
            new_employers_list.append(len(new_employers))
    return new_employers_list

def open_accounts_at_eom_list():
    '''
        This function will generate the Open Accounts At EOM column in the output
    '''
    open_account_count_list_per_eom = []
    for tier in get_distinct_tier_list():
        for eom in date_range()[2]:
            open_accounts = EMPLOYER_MASTER.loc[(EMPLOYER_MASTER["Tier"] == tier) & 
                                                (EMPLOYER_MASTER["Effective_From"] <= eom) & 
                                                (EMPLOYER_MASTER["Effective_To"] >= eom) &
                                                (EMPLOYER_MASTER["Status"] == "Open")]
            open_account_count_list_per_eom.append(len(open_accounts))
    return open_account_count_list_per_eom

def main():
    '''
        This function will generate the output
    '''
    data = {"Tier": tier_and_month_end_date_pairs()[0],
           "Month end date": tier_and_month_end_date_pairs()[1],
           "Num payments":payments()[0],
           "Amount of payments": payments()[1],
           "New employers": new_employers_list(),
           "Open employers at EOM": open_accounts_at_eom_list()}
    df = pd.DataFrame(data)
    df.to_csv('results_test.csv')

    
if __name__ == "__main__":
    main()

/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
